In [ ]:
!pip install transformers torch


In [ ]:
from google.colab import files
import pandas as pd

# Upload CSV from your computer
uploaded = files.upload()  # Select your CSV file

# Get uploaded filename
uploaded_filename = list(uploaded.keys())[0]

# Load CSV into a DataFrame
df = pd.read_csv(uploaded_filename, header=None, names=["tweet"])
df.head()


Saving twitter_validation.csv to twitter_validation.csv


,,,tweet
3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "yangheng/deberta-v3-base-absa-v1.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["negative", "neutral", "positive"]


Loading weights:   0%|          | 0/202 [00:00<?, ?it/s]

DebertaV2ForSequenceClassification LOAD REPORT from: yangheng/deberta-v3-base-absa-v1.1
Key                             | Status     |  | 
--------------------------------+------------+--+-
deberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
def aspect_sentiment(text, aspect):
    input_text = str(text) + " [SEP] " + aspect
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)

    sentiment = labels[torch.argmax(scores)]
    confidence = torch.max(scores).item()

    return sentiment, round(confidence, 3)


In [ ]:
aspects = ["Facebook", "CSGO", "WORD", "President"]


In [ ]:
# Create new columns for each aspect

# Select first 10 rows
df_subset = df.head(10)  # or df.iloc[:10]

# Apply ABSA only to the subset
for aspect in aspects:
    df_subset[f"{aspect}_sentiment"] = df_subset["tweet"].apply(lambda x: aspect_sentiment(x, aspect)[0])
    df_subset[f"{aspect}_confidence"] = df_subset["tweet"].apply(lambda x: aspect_sentiment(x, aspect)[1])

df_subset.head(10)



/tmp/ipython-input-3341829957.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[f"{aspect}_sentiment"] = df_subset["tweet"].apply(lambda x: aspect_sentiment(x, aspect)[0])
/tmp/ipython-input-3341829957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[f"{aspect}_confidence"] = df_subset["tweet"].apply(lambda x: aspect_sentiment(x, aspect)[1])
/tmp/ipython-input-3341829957.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,,,tweet,Facebook_sentiment,Facebook_confidence,CSGO_sentiment,CSGO_confidence,WORD_sentiment,WORD_confidence,President_sentiment,President_confidence
3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...,neutral,0.802,neutral,0.528,neutral,0.496,neutral,0.500
352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,neutral,0.729,neutral,0.784,neutral,0.717,neutral,0.725
8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,negative,0.630,negative,0.670,negative,0.927,negative,0.776
4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",negative,0.498,negative,0.909,negative,0.719,neutral,0.531
4433,Google,Neutral,Now the President is slapping Americans in the...,negative,0.761,negative,0.723,negative,0.776,negative,0.874
6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...,negative,0.526,negative,0.430,negative,0.530,negative,0.542
7925,MaddenNFL,Positive,Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...,positive,0.605,positive,0.599,positive,0.615,positive,0.601
11332,TomClancysRainbowSix,Positive,"Rocket League, Sea of Thieves or Rainbow Six: ...",positive,0.547,positive,0.567,positive,0.538,positive,0.568
1107,AssassinsCreed,Positive,my ass still knee-deep in Assassins Creed Odys...,positive,0.519,neutral,0.483,positive,0.481,positive,0.551
2069,CallOfDuty,Negative,FIX IT JESUS ! Please FIX IT ! What In the wor...,negative,0.521,negative,0.506,negative,0.507,negative,0.497


In [ ]:
df.to_csv("tweets_absa_results.csv", index=False)
print("Saved ABSA results as tweets_absa_results.csv")


Saved ABSA results as tweets_absa_results.csv


In [ ]:
print(aspects)
print(df.columns.tolist())



['Facebook', 'CSGO', 'WORD', 'President']
['tweet']


In [ ]:
import matplotlib.pyplot as plt

counts = df["tweet"].value_counts()
ax = counts.plot(kind="bar", color=["red", "gray", "green"])

plt.title("Facebook Aspect Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("Number of Tweets")

# Add percentage labels
for i, val in enumerate(counts):
    ax.text(i, val + 0.5, f'{val / counts.sum() * 100:.1f}%', ha='center')

plt.show()



/usr/local/lib/python3.12/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 128218 (\N{BOOKS}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.12/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 127881 (\N{PARTY POPPER}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.12/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 128077 (\N{THUMBS UP SIGN}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.12/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 10024 (\N{SPARKLES}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.12/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 127775 (\N{GLOWING STAR}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.12/dist-packages/IPyth

KeyboardInterrupt: 